In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
from sklearn import preprocessing
import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer


 tenemos que encontrar los mejores clientes (As per the Pareto Principle, 80% of outcomes result from 20% of all the causes of any given event. n business terms, we can say that 20% of customers contribute 80% share of the total revenue of a company) ya que son los que nos proporcionan el 80 por ciento de los ingresos.
 
tambien buscaremos los que estan a punto de irse o dejar la compañia (churn)

In [6]:
path = "C:/Users/gonza/OneDrive/Documentos/datascience-thebridge-master/ejercicios gonzalo/DATA"
df = pd.read_excel(os.path.join(path,"Online Retail.xlsx"))
df.head(3)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


In [14]:
# duplicados
print(f"el numero de duplicados son {df.duplicated().sum()}")
print(f"el porcentaje de duplicados son: {round((df.duplicated().sum() / len(df) ) * 100,2)}")

el numero de duplicados son 5268
el porcentaje de duplicados son: 0.97


In [15]:
# remove duplicate
df.drop_duplicates(inplace=True)

In [23]:
# contamos valores unicos de las siguientes columnas
lista_unique = ["StockCode","InvoiceNo","CustomerID"]
dict_unique = {i:len(df[i].unique()) for i in lista_unique}

In [27]:
pd.DataFrame(dict_unique,index=["quantity"])

,StockCode,InvoiceNo,CustomerID
quantity,4070,25900,4373


In [41]:
# vamos a ver cuantos pedidos por paises
country_orders = df.groupby("Country",as_index=False).agg({"InvoiceNo":"nunique"}).rename(columns={"InvoiceNo":"Orders"})
total_country_orders = country_orders["Orders"].sum(axis=0)
country_orders["%Orders"] = round(country_orders["Orders"] / total_country_orders * 100,2)


In [46]:
country_orders.sort_values(by="%Orders",ascending=False).head(10).style.background_gradient(cmap='Blues')

,Country,Orders,%Orders
36,United Kingdom,23494,90.710000
14,Germany,603,2.330000
13,France,461,1.780000
10,EIRE,360,1.390000
3,Belgium,119,0.460000
31,Spain,105,0.410000
24,Netherlands,101,0.390000
33,Switzerland,74,0.290000
0,Australia,69,0.270000
27,Portugal,71,0.270000


In [61]:
# total cancelaciones
invoices = df['InvoiceNo']
x = invoices.str.contains("C",regex=True)
x.fillna(0,inplace=True)
x = x.astype(int)

In [64]:
x.value_counts(normalize=True) * 100

0    98.276129
1     1.723871
Name: InvoiceNo, dtype: float64

In [63]:
# añadimos columna de canceladas
df["order_canceled"] = x

In [70]:
# vamos a trabajar con las no canceladas
df = df.loc[df["order_canceled"] == 0]

In [71]:
# hay que comprobar si hay facturas negativas que no constan como canceladas
df.loc[df["Quantity"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,order_canceled
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom,0
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom,0
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom,0
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom,0
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,NaN,United Kingdom,0
...,...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.0,NaN,United Kingdom,0
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,NaN,United Kingdom,0
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom,0
536908,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom,0


In [74]:
# vemos que cuando son cantidades negativas tambien customerid es Nan. Vamos a descartarlas y selecionar solamente UK
df_uk = df.loc[(df["Country"] == "United Kingdom") & (df["CustomerID"].notna())]
df_uk.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,order_canceled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0
